![buscar.png](./Data_piezas/buscar.png)

# Práctica Grupal: Diseño de Base de Datos

## <center>¡Busca al Proveedor!</center>

Para la práctica grupal, hemos creado nuestra propia versión del clásico *"Busca al asesino"*.  
Pero en este caso, el objetivo será **encontrar la información de un proveedor de un bote de pintura**.


---

👫 *María* y *Juan* quieren pintar la habitación de invitados. Han comprado un **bote de pintura** por internet pero no les convence el *color*.

📋 Para la devolución les solicitan que rellenen unos datos correspondientes al producto comprado pero solo tienen el **nombre del producto** y su **precio**.

<center>
<img src="./Data_piezas/buscar_2.png" alt="Description" width="90" height="120">
</center>

### Maria y Juan, necesitan encontrar el nombre del **proveedor** con su **dirección** y **ciudad**, la **fecha** en que compraron el producto y a que **categoría** pertenece el producto.


### Lo único que saben es que el producto se llama **Pintura plástica blanca** y les ha costado **22.00**

---

### <center> ¡Ven y ayuda a Juan y María a resolver este problema!</center>


### Modelo de datos
Primero hemos diseñado una base de datos sobre proveedores y piezas

![diseno.png](./Data_piezas/diseno.png)

Luego, hemos creado nuestra propia base de datos basada en el **Modelo de Datos**, y la hemos guardado con el nombre **`proveedores_piezas.db`**.

Para ello, utilizamos dos scripts:

- **`creacion_tabla.py`**: encargado de la creación de la tabla de datos.
- **`relleno_tabla.py`**: utilizado para completar la tabla con los datos necesarios.

In [5]:
!python ./Scripts/creacion_tabla.py
!python ./Scripts/relleno_tabla.py

## Usando nuestra base de datos, queremos ayudar a María y Juan 

In [10]:
import sqlite3
import pandas as pd

In [11]:
#Conexion a la base de datos
conexion = sqlite3.connect("proveedores_piezas.db")                        
cursor = conexion.cursor()

In [17]:
def sql_query(query):
    cursor.execute(query) 
    ans = cursor.fetchall()
    names = [d[0] for d in cursor.description]
    df = pd.DataFrame(ans,columns=names)
    return df

### Filtramos por precio `22.00`

In [13]:
query = '''   
SELECT * 
FROM pieza 
WHERE precio == 22.00
'''

sql_query(query)


,CODIGO_PIEZA,NOMBRE_PIEZA,COLOR,PRECIO,CODIGO_CATEGORIA
0,1008,Grava 20mm,Gris oscuro,22.0,207
1,1028,Pintura plástica blanca,Blanco mate,22.0,218


### ¡Encontramos dos productos!

Nos quedamos con los datos de la **pintura blanca**.

Vamos a buscar en primer lugar al **proveedor** con ayuda del **código de la pieza `1028`**.

In [15]:
query = '''  
SELECT a.codigo_pieza, a.nombre_pieza, b.id_proveedor
FROM pieza AS a
LEFT JOIN obtener_proveedor AS b
ON a.codigo_pieza = b.codigo_pieza
WHERE a.codigo_pieza = 1028
'''
sql_query(query)

,CODIGO_PIEZA,NOMBRE_PIEZA,ID_PROVEEDOR
0,1028,Pintura plástica blanca,28


### Con el **ID del proveedor** buscamos el **nombre** y la **dirección** que necesitan Maria y Juan

In [16]:
query = '''  
SELECT nombre, direccion, ciudad
FROM proveedores 
WHERE id_proveedor = 28
'''
sql_query(query)

,NOMBRE,DIRECCION,CIUDAD
0,La Pincelada Divertida,Calle Luz 36,León


In [32]:
df_proveedor = sql_query(query)
print(f"El proveedor es: {df_proveedor.NOMBRE.to_list()[0]}, \
con la dirección {df_proveedor.DIRECCION.to_list()[0]} \
en la ciudad de {df_proveedor.CIUDAD.to_list()[0]}.")

El proveedor es: La Pincelada Divertida, con la dirección Calle Luz 36 en la ciudad de León.


### Conocidos los datos del proveedor, solo nos queda saber la **categoria del producto** y la **fecha de compra**. 
Volvamos a utilizar el código de la pieza.

In [33]:
query = '''   
SELECT * 
FROM pieza AS a
LEFT JOIN categoria AS b
on a.codigo_categoria = b.codigo_categoria
WHERE precio == 22.00 AND nombre_pieza LIKe '%pintura%'
'''
sql_query(query)

,CODIGO_PIEZA,NOMBRE_PIEZA,COLOR,PRECIO,CODIGO_CATEGORIA,NOMBRE,CODIGO_CATEGORIA
0,1028,Pintura plástica blanca,Blanco mate,22.0,218,Pinturas y recubrimientos,218


### Ya sabemos que el bote de **Pintura plástica blanca** cuyo codigo de pieza es `1028` fue emitido por el proveedor `"La Picelada Divertida"` con dirección en `calle Luz 36, León`.
#### La categoria del producto se llama `Pinturas y recubrimientos`.
##### Vamos a por la fecha!!!!

In [34]:
query = '''   
SELECT * 
FROM pieza AS a
LEFT JOIN pieza_disponible AS b
on a.codigo_pieza = b.codigo_pieza
WHERE precio == 22.00 AND nombre_pieza LIKe '%pintura%'
'''
sql_query(query)

,CODIGO_PIEZA,NOMBRE_PIEZA,COLOR,PRECIO,CODIGO_CATEGORIA,CODIGO_PIEZA,FECHA,CANTIDAD
0,1028,Pintura plástica blanca,Blanco mate,22.0,218,1028,2025-03-07,95


In [37]:
df_fecha = sql_query(query)
print(f"La fecha de compra fue el: {df_fecha.FECHA.to_list()[0]}")

La fecha de compra fue el: 2025-03-07


## Fecha: `07/03/2025`  🎉🎉🎉🎉🎉

## <center> Maria y Juan quieren invitarte a una cena por haberles ayudado, es su manera de agradecerte tu amabilidad  🎉🎉🎉🎉🎉 🎉🎉🎉🎉🎉 </center>